In [1]:
import os

os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path
from src.Emotion_Detection import logger


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    testing_data: Path
    model_chkpt: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:




from src.Emotion_Detection.constants import *
from src.Emotion_Detection.utils.common import read_yaml, create_directories
import tensorflow as tf
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "train")
        testing_data= os.path.join(self.config.data_ingestion.unzip_dir, "test")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            testing_data=Path(testing_data),
            model_chkpt=Path(training.model_chkpt),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        logger.info("TrainingConfig successfully loaded")
        return training_config

In [4]:
import wandb

# # Log in with a different account
# wandb.login(relogin=True)
# run = wandb.init(
#     project = "intro-keras",
# )

In [23]:

from wandb.keras import WandbCallback,WandbMetricsLogger,WandbModelCheckpoint

In [6]:
from src.Emotion_Detection.utils.common import LogConfMatrix,LogResultsTable,plot_training_history


In [40]:
import os
import wandb

# Get the base path provided by W&B
base_path = wandb.run.dir

# Example absolute path


# Convert absolute path to relative path based on base_path
# relative_path = os.path.relpath(absolute_path, base_path)

# Save using relative path
wandb.save(base_path)


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


['c:\\Users\\lenovo\\Desktop\\Emotion-Detection\\wandb\\run-20240513_115255-qc2ms2xy\\files\\run-20240513_115255-qc2ms2xy\\files']

In [33]:


import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import time
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        train_datagenerator_kwargs = dict(
            rescale = 1./255,
        )



        train_datagenerator_ = tf.keras.preprocessing.image.ImageDataGenerator(
            **train_datagenerator_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=10,
                horizontal_flip=True,
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.2,
                **train_datagenerator_kwargs
            )
        else:
            train_datagenerator = train_datagenerator_



        self.train_generator = train_datagenerator.flow_from_directory(
                                                    self.config.training_data,  # Directory containing training data
                                                    class_mode="categorical",  # Classification mode for categorical labels
                                                    target_size=(224, 224),  # Resize input images to (224,224)
                                                    color_mode='rgb',  # Color mode for images (RGB)
                                                    shuffle=True,  # Shuffle training data
                                                    batch_size=self.config.params_batch_size,  # Batch size for training
                                                    subset='training'  # Subset of data (training)
                                                   )

        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                  rescale=1 / 255.  # Rescale pixel values to [0,1]
                                 )
        self.test_generator = test_datagen.flow_from_directory(
                                                  self.config.testing_data,  # Directory containing testing data
                                                  class_mode="categorical",  # Classification mode for categorical labels
                                                  target_size=(224, 224),  # Resize input images to (224,224)
                                                  color_mode="rgb",  # Color mode for images (RGB)
                                                  shuffle=False,  # Do not shuffle testing data
                                                  batch_size=self.config.params_batch_size  # Batch size for testing
                                                 )
        
                # Extract class labels for all instances in the training dataset
        classes = np.array(self.train_generator.classes)

        # Calculate class weights to handle imbalances in the training data
        # 'balanced' mode automatically adjusts weights inversely proportional to class frequencies
        class_weights = compute_class_weight(
            class_weight='balanced',  # Strategy to balance classes
            classes=np.unique(classes),  # Unique class labels
            y=classes  # Class labels for each instance in the training dataset
        )

        # Create a dictionary mapping class indices to their calculated weights
        self.class_weights_dict = dict(enumerate(class_weights))

        # Output the class weights dictionary
        print("Class Weights Dictionary:", self.class_weights_dict)



                # File path for the model checkpoint
        cnn_path =self.config.model_chkpt
        name = 'best_weights'
        chk_path = os.path.join(cnn_path, name)

        # Callback to save the model checkpoint
        checkpoint = ModelCheckpoint(filepath=chk_path,
                                    save_best_only=True,
                                    verbose=1,
                                    monitor='val_accuracy',
                                    mode = 'max')

        # Callback for early stopping
        earlystop = EarlyStopping(monitor = 'val_accuracy',
                                patience = 7,
                                restore_best_weights = True,
                                verbose=1)

        # Callback to reduce learning rate
        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                    factor=0.2,
                                    patience=2,
        #                             min_lr=0.00005,
                                    verbose=1)

        # Callback to log training data to a CSV file
        csv_logger = CSVLogger(os.path.join(cnn_path,'training.log'))
        wandb.login(relogin=True)
        run = wandb.init(
            project = "intro-keras",
        )
        # model_checkpoint = WandbModelCheckpoint(
        #     filepath=r'model_artifiact/best_model',
        #     monitor='val_accuracy',  # Monitor validation accuracy
        #     save_weights_only=False,  # Save entire model
        #     mode='max',  # Save when validation accuracy is maximized
        #     save_best_only=True,  # Save only the best model
        #     verbose=1,  # Verbosity level
        #     filename='best_model.h5',  # Filename for saved model
        #     log_weights=True  # Log histograms of the model's layer weights
        # )

        logger.info("wandb login succefully")
        wandb_callback = WandbCallback(
                    monitor='val_accuracy',  # Monitor validation accuracy
                    save_model='best',  # Save only the best model
                    save_model_path='best_model.h5'  # Custom name for the saved model
                )
        
        # Aggregating all callbacks into a list
        self.callbacks = [wandb_callback, earlystop, csv_logger,LogResultsTable(self.test_generator,self.model),LogConfMatrix(self.test_generator,self.model)]  # Adjusted as per your use-case
     
        logger.info("callbacks list succesfully inititated")
   
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        # Log in with a different account

        self.steps_per_epoch = (self.train_generator.samples // self.train_generator.batch_size)+1
        self.test_steps_epoch = (self.test_generator.samples // self.test_generator.batch_size)+1
        logger.info("training initated ....")
        history=self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=1,
            validation_steps=1,
            validation_data=self.test_generator,
            class_weight=self.class_weights_dict,
            callbacks=self.callbacks,
        )
        logger.info("saving model....")

        # Call the function to get the figure
        fig = plot_training_history(history)

        # Log the figure to Weights & Biases
        wandb.log({"training_history": fig})
        logger.info("wandb training history logged in ")

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [34]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-05-13 12:43:11,367: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-13 12:43:11,377: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-13 12:43:11,381: INFO: common: created directory at: artifacts]
[2024-05-13 12:43:11,384: INFO: common: created directory at: artifacts\training]
[2024-05-13 12:43:11,387: INFO: 4073233030: TrainingConfig successfully loaded]
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Class Weights Dictionary: {0: 1.0266046844269623, 1: 9.406618610747051, 2: 1.0010460615781582, 3: 0.5684387684387684, 4: 0.8260394187886635, 5: 0.8491274770777877, 6: 1.293372978330405}
[2024-05-13 12:43:30,190: INFO: 3461614742: wandb login succefully]


wandb: WARNING When using `save_best_only`, ensure that the `filepath` argument contains formatting placeholders like `{epoch:02d}` or `{batch:02d}`. This ensures correct interpretation of the logged artifacts.


[2024-05-13 12:43:30,354: INFO: 3461614742: callbacks list succesfully inititated]
[2024-05-13 12:43:30,369: INFO: 3461614742: training initated ....]
1/1 [==============================] - ETA: 0s - loss: 4.3384 - accuracy: 0.1562
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to model_artifiact\best_model
[2024-05-13 12:45:16,841: WARNING: save: Found untraced functions such as _update_step_xla, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 54). These functions will not be directly callable after loading.]
[2024-05-13 12:45:34,831: INFO: builder_impl: Assets written to: model_artifiact\best_model\assets]


wandb: Adding directory to artifact (.\model_artifiact\best_model)... Done. 8.6s


1/1 [==============================] - 133s 133s/step - loss: 4.3384 - accuracy: 0.1562 - val_loss: 7.7034 - val_accuracy: 0.0000e+00
[2024-05-13 12:45:47,618: INFO: 3461614742: saving model....]
[2024-05-13 12:45:47,620: INFO: 3461614742: wandb training history logged in ]
